In [1]:
import os
import sys
import numpy as np
import pandas as pd
from Utils.forest_cover import get_cover,green_cover_plot, green_cover_plot_smooth
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image, ImageFilter
import imutils
from pyimagesearch.colorlabeler import ColorLabeler
from Utils.read_coord import kml_coord
from Utils.sentinal_request import senti_api,bbox_cord

C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\fastkml\config.py:28: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")
C:\Users\DELL\anaconda3\envs\sentinal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = input('Enter kml file path: ')
print(file_path)
#C:\Users\Harith\EngenuityAI\SPE\SPE-development\sentinal\gAMPOLA TEST SITE 2.kml
if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()

        #print(lines)
else:
    print('The specified file does NOT exist')



# getting polygon coordinates
poly_coord = kml_coord(file_path)
#test_kml_files/Kolonnawa.kml

Enter kml file path: test_kml_files/Kolonnawa.kml
test_kml_files/Kolonnawa.kml
The file exists


In [3]:
#for original
bbx_coord,poly = bbox_cord(poly_coord)

In [4]:
img = cv2.imread('output/gren_cover_no_legend.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [5]:
#img = np.flip(img,0)
#hsv = np.flip(hsv,0)

In [6]:
#define colours

#green color 
lower_green = np.array([40,70,80])
upper_green = np.array([70,255,255])

#white
lower_white = np.array([255,255,255])
upper_white = np.array([255,255,255])


#add col mask
green = cv2.inRange(hsv, lower_green, upper_green)
white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img

In [7]:
def get_contours(img,hsv):
    #add col mask
    green = cv2.inRange(hsv, lower_green, upper_green)
    white = cv2.inRange(img, lower_white, upper_white)#use bgr instead of img
    #gree conoturs
    cnts_green = cv2.findContours(green,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_green = imutils.grab_contours(cnts_green)
    #white contours
    cnts_white = cv2.findContours(white,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnts_white = imutils.grab_contours(cnts_white)
    
    return cnts_green,cnts_white[1] #the boundry contour

In [13]:
def point_to_coord_boundry(cnt,bbx_coord):
    poly_arr = np.squeeze(cnt,1)
    #get the bbox
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])
    Xmin = df['lat'].min()
    Ymin = df['long'].min()
    Xmax = df['lat'].max()
    Ymax = df['long'].max()
    bbox = [Xmin,Ymin,Xmax,Ymax]
    
    #coordinate lengths
    coord_len = bbx_coord[2] - bbx_coord[0]
    coord_h = bbx_coord[3] - bbx_coord[1]
    
    #array point lengths
    img_len = bbox[2] - bbox[0]
    img_h = bbox[3] - bbox[1]
    
    #calculate ration diff
    ratio_len = coord_len/img_len
    ratio_h = coord_h/img_h
    
    #get the conversion 
    #coord = point min + (point-point min) * ratio
    lat = []
    long = []
    for index, row in df.iterrows():
        x = (row['lat'] - bbox[0]) * ratio_len
        y = (row['long'] - bbox[1]) * ratio_h
        #print(x)
        lat.append(bbx_coord[0]+x)
        long.append(bbx_coord[3]-y)
        
    poly = list(zip(lat, long))
    return poly,bbox

In [21]:
def point_to_coord_green(cnt,bbx_coord,bbox):
    poly_arr = np.squeeze(cnt,1)
    #get the bbox
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])

    #bbox = [Xmin,Ymin,Xmax,Ymax]
    
    #coordinate lengths
    coord_len = bbx_coord[2] - bbx_coord[0]
    coord_h = bbx_coord[3] - bbx_coord[1]
    
    #array point lengths
    img_len = bbox[2] - bbox[0]
    img_h = bbox[3] - bbox[1]
    
    #calculate ration diff
    ratio_len = coord_len/img_len
    ratio_h = coord_h/img_h

    #bbox,ratio_len,ratio_h = point_to_coord_boundry(bbox,bbx_coord)
    #get the conversion 
    #coord = point min + (point-point min) * ratio
    lat = []
    long = []
    for index, row in df.iterrows():
        x = (row['lat'] - bbox[0]) * ratio_len
        y = (row['long'] - bbox[1]) * ratio_h
        #print(x)
        ##can't do like this
        lat.append(bbx_coord[0]+x)
        long.append(bbx_coord[3]-y) #since the image location mapping difference with bbox coord
        
    poly = list(zip(lat, long))
    return poly

In [22]:
greens,white = get_contours(img,hsv)

In [23]:
boundry_poly,bbox = point_to_coord_boundry(white,bbx_coord)
boundry_poly

[(79.88636855615572, 6.939900574336073),
 (79.8864204913662, 6.939952695830147),
 (79.88657629699766, 6.939952695830147),
 (79.88662823220814, 6.939900574336073),
 (79.88662823220814, 6.9397963313479245),
 (79.88688790826058, 6.9397963313479245),
 (79.88693984347105, 6.939744209853851),
 (79.887147584313, 6.939744209853851),
 (79.88719951952348, 6.939692088359776),
 (79.88719951952348, 6.939639966865703),
 (79.88740726036542, 6.939639966865703),
 (79.88745919557591, 6.939587845371628),
 (79.88745919557591, 6.939535723877555),
 (79.88766693641784, 6.939535723877555),
 (79.88771887162834, 6.93948360238348),
 (79.88771887162834, 6.939431480889406),
 (79.88792661247028, 6.939431480889406),
 (79.88797854768076, 6.939379359395332),
 (79.88813435331221, 6.939379359395332),
 (79.8881862885227, 6.939327237901258),
 (79.8881862885227, 6.939275116407184),
 (79.88839402936465, 6.939275116407184),
 (79.88844596457513, 6.93922299491311),
 (79.88844596457513, 6.939170873419036),
 (79.88865370541707, 

In [24]:
greens = sorted(greens, key=cv2.contourArea, reverse= True)

In [34]:
green_coords = []
for green in greens:
    #print(green.shape)
    g_coord = point_to_coord_green(green,bbx_coord,bbox)
    green_coords.append(g_coord)

In [26]:
green_coords

[[(79.90184524888033, 6.927756266216807),
  (79.90179331366984, 6.927704144722734),
  (79.90153363761742, 6.927704144722734),
  (79.90142976719645, 6.9275999017345855),
  (79.90132589677548, 6.9275999017345855),
  (79.90132589677548, 6.92728717277014),
  (79.901273961565, 6.9272350512760665),
  (79.90111815593355, 6.9272350512760665),
  (79.90106622072305, 6.92728717277014),
  (79.90106622072305, 6.927391415758288),
  (79.90096235030208, 6.927391415758288),
  (79.9009104150916, 6.927443537252363),
  (79.9009104150916, 6.9275999017345855),
  (79.90070267424966, 6.9275999017345855),
  (79.90059880382869, 6.927495658746437),
  (79.90059880382869, 6.927443537252363),
  (79.90065073903918, 6.927391415758288),
  (79.90065073903918, 6.927078686793844),
  (79.90059880382869, 6.92702656529977),
  (79.90059880382869, 6.926922322311622),
  (79.90059880382869, 6.92702656529977),
  (79.90065073903918, 6.927078686793844),
  (79.90070267424966, 6.927078686793844),
  (79.90075460946014, 6.927026565299

In [27]:
from Utils.map_view import map_show

In [28]:
map_show(boundry_poly)

In [66]:
greens[0].shape

(922, 1, 2)

In [33]:
map_show(green_coords[3])